**ПОДГОТОВКА ОКРУЖЕНИЯ**


In [1]:
# Импортируем библиотеки, которые нам понадобятся для работы

# Pandas основная библиотека для анализа данных, работы с датафреймами
# Logging библиотека для вывода информационных сообщений о ходе выполнения программы

import pandas as pd
import logging

**Настраиваем систему логирования**

In [2]:
logging.basicConfig(
    level=logging.INFO, # Указываем, что хотим видеть сообщения уровня INFO и выше
    format="%(asctime)s [%(levelname)s] %(message)s" # а также задаем формат сообщений (время, уровень важности и само сообщение)
)

logger = logging.getLogger(__name__) # Создаем конкретный логгер для нашего скрипта, чтобы потом выводить сообщения о ходе работы

# WINNERS DATA PIPELINE

**ПРОЕКТИРОВАНИЕ МЕТАДАННЫХ (ЭТАП EXTRACT)**

In [3]:
# Года проведения The International (2020 год пропущен, так как турнир был отменен из-за пандемии)
years_df = pd.DataFrame({
    'year': [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021]
})

# Базовая ссылка на csv по победителям и странам участников (им нужно добавить только год в конце)

# base_winners - данные о победителях и призовых (кто сколько занял и получил)
base_winners = "https://raw.githubusercontent.com/alexandraryzvanova-sketch/DWH_quality/main/data/raw/winners/PrizePoolFinalStandings"
# base_country - данные о представительстве стран в турнире (из каких стран игроки)
base_country = "https://raw.githubusercontent.com/alexandraryzvanova-sketch/DWH_quality/main/data/raw/country/CountryRepresentation"

# Создаю таблицу по годам и ссылкам на нужные мне csv
metadata_df = pd.DataFrame({
    "year": years_df['year'],
    "winners_csv": [f"{base_winners}{y}.csv" for y in years_df['year']],
    "country_csv": [f"{base_country}{y}.csv" for y in years_df['year']],
})

# Цель: зафиксировать года проведения TI и ссылки на исходные данные для последующей загрузки и анализа качества данных
logger.info("Metadata prepared")
metadata_df.head()

,year,winners_csv,country_csv
0,2011,https://raw.githubusercontent.com/alexandraryz...,https://raw.githubusercontent.com/alexandraryz...
1,2012,https://raw.githubusercontent.com/alexandraryz...,https://raw.githubusercontent.com/alexandraryz...
2,2013,https://raw.githubusercontent.com/alexandraryz...,https://raw.githubusercontent.com/alexandraryz...
3,2014,https://raw.githubusercontent.com/alexandraryz...,https://raw.githubusercontent.com/alexandraryz...
4,2015,https://raw.githubusercontent.com/alexandraryz...,https://raw.githubusercontent.com/alexandraryz...


**ТРАНСФОРМАЦИЯ ДАННЫХ (ЭТАП TRANSFORM)**


In [4]:
# Формируя таблицу winners мы сталкиваемся с проблемой несогласованности схем данных: разные годы турнира одни и те же колонки называются по-разному
# Например, колонка с названием команды может называться "Team Name", "Team"
# Задача: привести все к единому стандарту

def normalize_columns(df):

    """
    Функция для приведения названий колонок к единому формату

    Аргументы:
        df: исходный датафрейм с "сырыми" данными за конкретный год

    Возвращает:
        df: датафрейм с приведенными к единому стандарту названиями колонок

    Принцип работы:
    Проверяем, какие названия колонок есть в исходном файле, и создаем новые
    колонки со стандартизированными именами: team_name, prize_usd, prize_percent
    """

    df = df.copy() # Создаем копию датафрейма, чтобы не изменять оригинальные данные

    if 'Team Name' in df.columns:
        df['team_name'] = df['Team Name']
    elif 'Team' in df.columns:
        df['team_name'] = df['Team']

    if '$USD' in df.columns:
        df['prize_usd'] = df['$USD']
    elif 'Price' in df.columns:
        df['prize_usd'] = df['Price']

    if 'PricePoolPercent' in df.columns:
        df['prize_percent'] = df['PricePoolPercent']
    elif 'Pcnt' in df.columns:
        df['prize_percent'] = df['Pcnt']
    elif 'Percent' in df.columns:
        df['prize_percent'] = df['Percent']

    return df

In [5]:
logger.info("Starting extraction of winners data") # Логируем начало процесса загрузки

winners = [] # Создаем пустой список, в который будем собирать данные за все года

for _, row in metadata_df.iterrows():
    df = pd.read_csv(row['winners_csv']) # Загружаю CSV
    df = normalize_columns(df) # Нормализуем весь df

    df_top3 = df.head(3).copy() # Берем топ 3, копируем, чтобы работать с копией, чтобы изменения в df_top3 не влияли на исходный df
    df_top3['year'] = row['year'] # Добавляем год

    winners.append(df_top3) # Добавляем обработанный датафрейм в наш список

df_winners_top3 = pd.concat(winners, ignore_index=True) # Объединяем все датафреймы из списка в один большой датафрейм, создаем новую нумерацию строк

logger.info(f"Loaded {len(df_winners_top3)} records") # Логируем результат: сколько всего записей загружено

df_winners_top3.head()

,Unnamed: 0,Place,Price,PricePoolPercent,Team Name,team_name,prize_usd,prize_percent,year,$USD,Pcnt,Team,Percent
0,0,1st,"$1,000,000",62.5%,Natus Vincere,Natus Vincere,"$1,000,000",62.5%,2011,NaN,NaN,NaN,NaN
1,1,2nd,"$250,000",15.625%,EHOME,EHOME,"$250,000",15.625%,2011,NaN,NaN,NaN,NaN
2,2,3rd,"$150,000",9.375%,Scythe Gaming,Scythe Gaming,"$150,000",9.375%,2011,NaN,NaN,NaN,NaN
3,0,1st,"$1,000,000",62.5%,Invictus Gaming,Invictus Gaming,"$1,000,000",62.5%,2012,NaN,NaN,NaN,NaN
4,1,2nd,"$250,000",15.625%,Natus Vincere,Natus Vincere,"$250,000",15.625%,2012,NaN,NaN,NaN,NaN


ФОРМИРОВАНИЕ ФАКТ-ТАБЛИЦЫ

In [6]:
winners_top3 = df_winners_top3[
    ['year', 'Place', 'team_name', 'prize_usd', 'prize_percent']
].copy() # Отбираем только нужные колонки для будущей таблицы, + копию, чтобы не портить исходные данные

ОЧИСТКА И ПРИВЕДЕНИЕ ТИПОВ ДАННЫХ

In [7]:
print("\nТипы данных:")
print(winners_top3.dtypes)


Типы данных:
year              int64
Place            object
team_name        object
prize_usd        object
prize_percent    object
dtype: object


In [11]:
# ОЧИСТКА КОЛОНКИ С ПРИЗОВЫМИ (prize_usd)
# Удаление знака доллара и запятых
winners_top3['prize_usd'] = winners_top3['prize_usd'].astype(str).str.replace('[$,]', '', regex=True)
# ОЧИСТКА КОЛОНКИ С ПРОЦЕНТАМИ (prize_percent)
# # Удаление знака процента и запятых
winners_top3['prize_percent'] = winners_top3['prize_percent'].astype(str).str.replace('[%,]', '', regex=True)

# ПРЕОБРАЗОВАНИЕ В ЧИСЛОВЫЕ ТИПЫ
winners_top3['prize_usd'] = pd.to_numeric(winners_top3['prize_usd'])
winners_top3['prize_percent'] = pd.to_numeric(winners_top3['prize_percent'])

logger.info("Transformation completed") # Логируем успешное завершение трансформации
print(winners_top3.dtypes)

winners_top3.head()

year               int64
Place             object
team_name         object
prize_usd          int64
prize_percent    float64
dtype: object


,year,Place,team_name,prize_usd,prize_percent
0,2011,1st,Natus Vincere,1000000,62.500
1,2011,2nd,EHOME,250000,15.625
2,2011,3rd,Scythe Gaming,150000,9.375
3,2012,1st,Invictus Gaming,1000000,62.500
4,2012,2nd,Natus Vincere,250000,15.625


**ПРОВЕРКА КАЧЕСТВА ДАННЫХ ( ЭТАП DATA QUALITY)**

In [12]:
def validate_winners(df):

    """
    Функция для комплексной проверки качества данных о победителях.

    Что проверяем:
    1. Отсутствие NULL-значений в критически важных полях
    2. Корректность форматов данных (места должны быть 1st, 2nd, 3rd)
    3. Бизнес-правила (призовые не могут быть отрицательными)
    4. Уникальность ключей (в одном году не может быть двух первых мест)
    5. Полноту данных (в каждом году должно быть ровно 3 призовых места)

    Аргументы:
        df: датафрейм с данными о победителях (после трансформации)

    Возвращает:
        None, но если проверки не пройдены - вызывается исключение ValueError
    """

    errors = [] # Создается пустой список, куда будут складываться все найденные ошибки

    # ПРОВЕРКА 1: Отсутствие NULL-значений

    if df['team_name'].isna().any():
        errors.append("team_name contains NULLs") # Проверка на пустые значения в колонке "team_name"

    if df['year'].isna().any():
        errors.append("year contains NULLs") # Проверка на пустые значения в колонке "year"

    if df['Place'].isna().any():
        errors.append("Place contains NULLs") # Проверка на пустые значения в колонке "Place"

    # ПРОВЕРКА 2: Корректность значений места

    valid_places = {'1st', '2nd', '3rd'}
    invalid_places = df.loc[~df['Place'].isin(valid_places), 'Place'].unique() # Проверяем, что у каждого победителя указано место (1st, 2nd, 3rd)

    if len(invalid_places) > 0:
        errors.append(f"Invalid Place values found: {list(invalid_places)}") # Если есть неправильные места добавляем их список в ошибки

    # ПРОВЕРКА 3: Бизнес-правила

    if (df['prize_usd'] < 0).any():
        errors.append("Negative prize_usd detected") # Проверка отрицательных призовых

    # ПРОВЕРКА 4: Уникальность записей

    dup = df.duplicated(subset=['year', 'Place']).sum()
    if dup > 0:
        errors.append(f"Duplicate records found: {dup}") # Проверка дубликатов, уникальности ключей (год + место)

    # ПРОВЕРКА 5: Полнота данных

    place_counts = df.groupby('year')['Place'].nunique()
    broken_years = place_counts[place_counts != 3]  # Проверка количества мест в каждом году, должно быть ровно 3

    if not broken_years.empty:
        errors.append(f"Years with incorrect number of places: {broken_years.to_dict()}")

    if errors:
        logger.error("DATA QUALITY VALIDATION FAILED")
        for err in errors:
            logger.error(err)
        raise ValueError("Data quality checks failed")  # Обработка найденных ошибок, если список ошибок не пустой, то выйдет ошибка

    logger.info("Data Quality validation passed")

ЗАПУСК ПРОВЕРКИ КАЧЕСТВА

In [13]:
validate_winners(winners_top3)

**ИССЛЕДОВАТЕЛЬСКИЙ АНАЛИЗ ДАННЫХ (EDA)**

In [14]:
print("\nСтатистика по призовым:")
print(winners_top3[['prize_usd', 'prize_percent']].describe())


Статистика по призовым:
          prize_usd  prize_percent
count  3.000000e+01      30.000000
mean   4.157821e+06      24.550000
std    4.575513e+06      17.725627
min    1.500000e+05       9.000000
25%    1.009444e+06      10.500000
50%    2.768734e+06      15.625000
75%    4.884499e+06      44.000000
max    1.820830e+07      62.500000


ЧТО ПОКАЗЫВАЕТ СТАТИСТИКА:

Количество (count): в обоих случаях 30 значений, что соответствует числу строк, все данные на месте, ни одна запись не потерялась при трансформации

*Среднее (mean):*
- prize_usd: 4.16 миллиона долларов - средний приз за всю историю турниров,
- prize_percent: 24.55% - в среднем команды получают четверть призового фонда

*Стандартное отклонение (std):* очень большие значения (4.6 млн и 17.8%) - это говорит о высокой вариативности, призовые сильно менялись от года к году, так как призовой фонд рос

*Минимум (min):*
- prize_usd: 150 тысяч долларов - самый маленький приз за всю историю,
- prize_percent: 9% - минимальная доля призового фонда (третье место в ранних турнирах)

*Квартильные значения (25%, 50%, 75%):*

Если посмотреть на значения от меньших к большим, видно, как резко увеличивались выигрыши
   - 25% → 50%: рост с 1.0 млн до 2.8 млн (+180%)
   - 50% → 75%: рост с 2.8 млн до 4.9 млн (+175%)
   - 75% → max: рывок с 4.9 млн до 18.2 млн (+370%)

Данные показывают **экспоненциальный рост призовых** за 10 лет.  
Если в начале истории TI призовые измерялись сотнями тысяч, то позже уже десятками миллионов.

**СОХРАНЕНИЕ ПРОМЕЖУТОЧНЫХ РЕЗУЛЬТАТОВ (LOAD PREVIEW)**

In [ ]:
output_path = "/content/winners_top3.csv" # /content/ - это стандартная папка в Google Colab
winners_top3.to_csv(output_path, index=False)

logger.info(f"Data mart file saved to {output_path}") # Логируем успешное сохранение с указанием пути к файлу

# COUNTRY DATA PIPELINE

**ЭТАП EXTRACT**

In [16]:
country_raw = [] # Создаем пустой список для сбора "сырых" данных за все года

for _, row in metadata_df.iterrows():
    year = row['year']
    url = row['country_csv']

    df = pd.read_csv(url)
    df['year'] = year  # добавляем год для дальнейшей аналитики

    country_raw.append(df)

country_df = pd.concat(country_raw, ignore_index=True)

# Выводим информацию о размере полученного датафрейма
# .shape возвращает кортеж (количество строк, количество колонок)
print("Raw Country Data Shape:", country_df.shape)
country_df.head()

Raw Country Data Shape: (215, 6)


,Unnamed: 0,Country,No. of Players,Players,year,Representation
0,0,China,19.0,"Awoke,Kabu,DD,Hao,Faith,SanSheng,Zhou,xiao8,Fe...",2011,NaN
1,1,Denmark,9.0,"MiSeRy,MaNia,PlaymatE,Maelk,syndereN,AngeL,MiG...",2011,NaN
2,2,Russia,7.0,"LighTofHeaveN,Azen,NS,Dread,ARS-ART,Santa,God",2011,NaN
3,3,France,5.0,"Maldejambes,VladTepes,Sockshka,Pseudo,Ph0eniX",2011,NaN
4,4,Malaysia,5.0,"SilverCross,Net,Sharky,xiaoling,kYxY",2011,NaN


**ЭТАП TRANSFORM**

In [19]:
# Разбиваем строку с игроками в список
country_df['player_name'] = country_df['Players'].str.split(',')

# Превращаем список в строки (explode)
country_exploded_df = country_df.explode('player_name')

print("После explode:", country_exploded_df.shape)
country_exploded_df.head()

После explode: (865, 7)


,Unnamed: 0,Country,No. of Players,Players,year,Representation,player_name
0,0,China,19.0,"Awoke,Kabu,DD,Hao,Faith,SanSheng,Zhou,xiao8,Fe...",2011,NaN,Awoke
0,0,China,19.0,"Awoke,Kabu,DD,Hao,Faith,SanSheng,Zhou,xiao8,Fe...",2011,NaN,Kabu
0,0,China,19.0,"Awoke,Kabu,DD,Hao,Faith,SanSheng,Zhou,xiao8,Fe...",2011,NaN,DD
0,0,China,19.0,"Awoke,Kabu,DD,Hao,Faith,SanSheng,Zhou,xiao8,Fe...",2011,NaN,Hao
0,0,China,19.0,"Awoke,Kabu,DD,Hao,Faith,SanSheng,Zhou,xiao8,Fe...",2011,NaN,Faith


ОЧИСТКА ДАННЫХ

In [20]:
# Отбираем только нужные колонки
# .copy() создает копию, чтобы изменения не затронули исходный датафрейм
country_clean_df = country_exploded_df[['Country', 'player_name', 'year']].copy()

# Убираем лишние пробелы в именах игроков
# .str.strip() удаляет пробелы в начале и конце каждой строки
country_clean_df['player_name'] = country_clean_df['player_name'].str.strip()

# Удаляем дубликаты (один игрок не должен повторяться)
country_clean_df = country_clean_df.drop_duplicates().reset_index(drop=True)

print("Clean dataset shape:", country_clean_df.shape)
country_clean_df.head()

Clean dataset shape: (865, 3)


,Country,player_name,year
0,China,Awoke,2011
1,China,Kabu,2011
2,China,DD,2011
3,China,Hao,2011
4,China,Faith,2011


**ПРОВЕРКА КАЧЕСТВА ДАННЫХ ( ЭТАП DATA QUALITY)**

ПЕРВИЧНЫЙ АНАЛИЗ СХЕМЫ ОЧИЩЕННЫХ ДАННЫХ

In [22]:
print("Data types:")
print(country_clean_df.dtypes)

print("\nMissing values:")
print(country_clean_df.isnull().sum())

Data types:
Country        object
player_name    object
year            int64
dtype: object

Missing values:
Country        0
player_name    0
year           0
dtype: int64


In [23]:
def validate_country(df):

    """
    Функция для проверки качества данных о странах и игроках.

    Что проверяем:
    1. Отсутствие NULL-значений в критических полях
    2. Пустые строки (актуально после explode)
    3. Дубликаты (один игрок из одной страны в один год не должен повторяться)
    4. Бизнес-правила (страна не может быть числом)

    Аргументы:
        df: датафрейм с данными о странах и игроках
    """

    errors = [] # Создаем пустой список для сбора ошибок

    # 1. Проверка NULL

    if df['Country'].isna().any():
        errors.append("Country contains NULLs")

    if df['player_name'].isna().any():
        errors.append("player_name contains NULLs")

    if df['year'].isna().any():
        errors.append("year contains NULLs")

    # 2. Проверка пустые строки

    if (df['player_name'] == "").any():
        errors.append("Empty player names detected")

    # 3. Проверка дублей (гранулярность)

    dup = df.duplicated(subset=['Country', 'player_name', 'year']).sum()
    if dup > 0:
        errors.append(f"Duplicate player-country-year records: {dup}")

    # 4. Бизнес-правило: страна не должна быть числом

    if df['Country'].str.isnumeric().any():
        errors.append("Invalid Country values detected")

    # Итог

    if errors:
        logger.error("COUNTRY DATA VALIDATION FAILED")
        for err in errors:
            logger.error(err)
        raise ValueError("Country data quality checks failed")

    # Если все проверки пройдены

    logger.info("Country validation passed successfully")


In [24]:
# ЗАПУСК ПРОВЕРКИ
validate_country(country_clean_df)

**МИНИ-АНАЛИЗ ДАННЫХ (EDA)**

In [25]:
print("\nТоп-10 стран по количеству игроков в призерах TI:")
# .value_counts() считает, сколько раз встречается каждая страна
# .head(10) показывает только первые 10 строк (топ-10 стран)
print(
    country_clean_df['Country']
    .value_counts()
    .head(10)
)


Топ-10 стран по количеству игроков в призерах TI:
Country
China            223
Malaysia          64
Russia            56
Sweden            52
Ukraine           50
United States     50
Philippines       48
Denmark           34
Canada            33
Germany           32
Name: count, dtype: int64


**СОХРАНЕНИЕ ПРОМЕЖУТОЧНЫХ РЕЗУЛЬТАТОВ (LOAD PREVIEW)**

In [ ]:
output_path = "/content/country.csv"
country_clean_df.to_csv(output_path, index=False)

logger.info(f"Data mart file saved to {output_path}")

# HERO PICKS DATA PIPELINE

**ЭТАП EXTRACT**

In [26]:
# Данные по пикам героев представлены в двух разных форматах, зависящих от года проведения турнира:

# 2011-2015: один формат файлов (MostCommonHeroPicks), за 2013 год отсутствуют данные
# Упрощённая структура (3 столбца), за 2020 год отсутствуют данные

# 2016-2021: другой формат (HeroStatistics)
#Расширенная структура (много метрик)

# Для корректной обработки данные разделяются на OLD и NEW группы.

# Определяем года для каждого формата
old_years = [2011, 2012, 2014, 2015]
new_years = [2016, 2017, 2018, 2019, 2021]

# Базовые URL для каждого формата
base_old = "https://raw.githubusercontent.com/alexandraryzvanova-sketch/DWH_quality/main/data/raw/pick/MostCommonHeroPicks"
base_new = "https://raw.githubusercontent.com/alexandraryzvanova-sketch/DWH_quality/main/data/raw/pick/HeroStatistics"

ЭТАП EXTRACT ДЛЯ СТАРОГО ФОРМАТА (2011-2015)

In [27]:
old_raw = [] # Создаем список для сбора "сырых" данных

for year in old_years: # Загружаем данные за каждый год старого формата
    url = f"{base_old}{year}.csv"  # Формируем URL: базовый адрес + год + .csv
    df = pd.read_csv(url) # Загружаем CSV
    df['year'] = year # Добавляем колонку с годом (в исходных данных ее нет)
    old_raw.append(df) # Добавляем в список

df_old_raw = pd.concat(old_raw, ignore_index=True) # Объединяем все загруженные данные в один датафрейм

print("OLD raw shape:", df_old_raw.shape) # смотрим размер: (строк, колонок)
df_old_raw.head()

OLD raw shape: (39, 5)


,Unnamed: 0,Hero,Number of times picked,year,Times Picked
0,0,Vengeful Spirit,33.0,2011,NaN
1,1,Windrunner,29.0,2011,NaN
2,2,Earthshaker,28.0,2011,NaN
3,3,Beastmaster,24.0,2011,NaN
4,4,Mirana,22.0,2011,NaN


**ЭТАП TRANSFORM для OLD (2011-2015)**

In [28]:
def normalize_old_picks(df):

    """
    Приводит данные старого формата (2011-2015) к единому стандарту.

    Проблема: в разные годы колонка с количеством пиков называлась:
    - "Number of times picked"
    - "Times Picked"

    Решение: проверяем оба варианта и создаем стандартную колонку 'times_picked'
    """

    df = df.copy() # Создаем копию, чтобы не изменять исходные данные

    if 'Number of times picked' in df.columns:
        df['times_picked'] = df['Number of times picked']
    elif 'Times Picked' in df.columns:
        df['times_picked'] = df['Times Picked']
    else:
        df['times_picked'] = 0  # fallback защита пайплайна

    df = df[['Hero', 'times_picked', 'year']]

    return df

ПРИМЕНЯЕМ НОРМАЛИЗАЦИЮ К СТАРЫМ ДАННЫМ

In [29]:
old_clean = [] # Создаем список для чистых данных

for year in old_years: # Загружаем и нормализуем данные за каждый год
    url = f"{base_old}{year}.csv"
    df = pd.read_csv(url)
    df['year'] = year
    df = normalize_old_picks(df) # применяем функцию
    old_clean.append(df)

df_old = pd.concat(old_clean, ignore_index=True) # Объединяем все нормализованные данные

print("OLD normalized shape:", df_old.shape)
df_old.head()

OLD normalized shape: (39, 3)


,Hero,times_picked,year
0,Vengeful Spirit,33,2011
1,Windrunner,29,2011
2,Earthshaker,28,2011
3,Beastmaster,24,2011
4,Mirana,22,2011


**ЭТАП EXTRACT ДЛЯ НОВОГО ФОРМАТА (2016-2021)**

In [30]:
new_raw = [] # Создаем список для новых данных

for year in new_years: # Загружаем данные за каждый год нового формата
    url = f"{base_new}{year}.csv"
    df = pd.read_csv(url)
    df['year'] = year
    new_raw.append(df)

df_new = pd.concat(new_raw, ignore_index=True) # Объединяем

print("NEW raw shape:", df_new.shape)
df_new.head()

NEW raw shape: (548, 16)


,Unnamed: 0,Hero,Times Picked,Win,Loss,WR%,TimesPicked%,Times Picked(Radiant side),Win(Radiant side),Loss(Radiant side),WR%(Radiant side),Times Picked(Dire side),Win(Dire side),Loss(Dire side),WR%(Dire side),year
0,0,Mirana,81,44,37,54.32%,50.94%,37,21,16,56.76%,44,23,21,52.27%,2016
1,1,Batrider,71,34,37,47.89%,44.65%,44,22,22,50.00%,27,12,15,44.44%,2016
2,2,Shadow Demon,60,35,25,58.33%,37.74%,31,20,11,64.52%,29,15,14,51.72%,2016
3,3,Elder Titan,49,32,17,65.31%,30.82%,20,10,10,50.00%,29,22,7,75.86%,2016
4,4,Ogre Magi,49,21,28,42.86%,30.82%,23,11,12,47.83%,26,10,16,38.46%,2016


**ЭТАП TRANSFORM для NEW (2016-2021)**

In [31]:
# В новом формате колонка уже называется "Times Picked"
# Просто переименую ее в единый стандарт

df_new.rename(columns={'Times Picked': 'times_picked'}, inplace=True) # inplace=True означает "изменить существующий датафрейм", а не создавать копию

df_new = df_new[['Hero', 'times_picked', 'year']] # Оставляем только нужные колонки (как в старом формате)

print("NEW transformed shape:", df_new.shape)
df_new.head()

NEW transformed shape: (548, 3)


,Hero,times_picked,year
0,Mirana,81,2016
1,Batrider,71,2016
2,Shadow Demon,60,2016
3,Elder Titan,49,2016
4,Ogre Magi,49,2016


ОБЪЕДИНЯЕМ СТАРЫЕ И НОВЫЕ ДАННЫЕ

In [32]:
hero_picks = pd.concat([df_old, df_new], ignore_index=True)

print("Final dataset shape:", hero_picks.shape)
hero_picks.head()

Final dataset shape: (587, 3)


,Hero,times_picked,year
0,Vengeful Spirit,33,2011
1,Windrunner,29,2011
2,Earthshaker,28,2011
3,Beastmaster,24,2011
4,Mirana,22,2011


**ПРОВЕРКА КАЧЕСТВА ДАННЫХ ( ЭТАП DATA QUALITY)**

In [33]:
def validate_hero_picks(df):

    """
    Комплексная проверка качества данных о пиках героев.

    Проверяемые аспекты:
    1. Отсутствие пропущенных значений (NULL)
    2. Корректность бизнес-показателей (неотрицательность)
    3. Правильность типов данных
    4. Уникальность записей (гранулярность)
    5. Логическая полнота данных (если турнир был - должны быть пики)
    """

    errors = [] # Создаем пустой список для сбора ошибок

    # 1. Проверка NULLs
    if df['Hero'].isna().any():
        errors.append("Hero contains NULLs")

    if df['year'].isna().any():
        errors.append("year contains NULLs")

    # 2. Проверка пустые строки
    if (df['times_picked'] < 0).any():
        errors.append("Negative times_picked detected")

    # 3. Проверка типа данных
    if not pd.api.types.is_numeric_dtype(df['times_picked']):
        errors.append("times_picked must be numeric")

    # 4. Проверка гранулярности (герой + год уникальны)
    dup = df.duplicated(subset=['Hero', 'year']).sum()
    if dup > 0:
        errors.append(f"Duplicate hero-year rows detected: {dup}")

    # 5. Логическая полнота данных: если турнир был должен быть хотя бы 1 пик
    zero_pick_years = df.groupby('year')['times_picked'].sum()
    zero_pick_years = zero_pick_years[zero_pick_years == 0]

    if not zero_pick_years.empty:
        errors.append(f"Years with zero picks detected: {zero_pick_years.index.tolist()}")

    # Итог
    if errors:
        logger.error("HERO PICKS VALIDATION FAILED")
        for err in errors:
            logger.error(err)
        raise ValueError("Hero Picks data quality checks failed")

    logger.info("Hero Picks validation passed successfully")


In [34]:
#ЗАПУСК ПРОВЕРКИ
validate_hero_picks(hero_picks)

**БЫСТРЫЙ АНАЛИЗ ПОЛУЧЕННЫХ ДАННЫХ**

In [35]:
print(hero_picks['times_picked'].describe())

count    587.000000
mean      19.943782
std       22.708238
min        1.000000
25%        5.000000
50%       13.000000
75%       26.000000
max      192.000000
Name: times_picked, dtype: float64


ЧТО ПОКАЗЫВАЕТ СТАТИСТИКА:

- Count: 587 записей - это все комбинации "герой + год" за всю историю турниров. Каждая запись показывает, сколько раз конкретного героя пикнули за весь турнир.


- Mean: В среднем героя пикают 20 раз за турнир.

Большой разброс
- Min: 1 есть герои с 1 пиком
- Max: 192 есть герои с 192 пиком

- Половина героев пикается от 5 до 26 раз.
- А 25% самых популярных героев пикаются больше 26 раз.

Это логично для Dota 2: в каждом патче есть сильные герои, которых берут постоянно, и слабые, которые почти не появляются на про-сцене.

**СОХРАНЕНИЕ ПРОМЕЖУТОЧНЫХ РЕЗУЛЬТАТОВ (LOAD PREVIEW)**

In [ ]:
output_path = "/content/hero_picks.csv"
hero_picks.to_csv(output_path, index=False)

logger.info(f"Data mart file saved to {output_path}")

# TEAMS DATA PIPELINE

**ЭТАП EXTRACT**

In [36]:
# Базовый URL, где хранятся CSV-файлы с составами команд
base_url = "https://raw.githubusercontent.com/alexandraryzvanova-sketch/DWH_quality/main/data/raw/teams/"

dfs = [] # Создаем пустой список для сбора данных

# Проходим по всем уникальным командам из таблицы победителей
# .drop_duplicates() убирает повторы (одна команда может побеждать несколько раз)
for _, row in winners_top3[['team_name', 'year']].drop_duplicates().iterrows():
    team = row['team_name'] # Название команды
    year = row['year'] # Год турнира

    # Заменяем пробелы в названии команды на %20 для корректного URL
    # URL-кодирование: пробел в интернете пишется как %20
    team_url = team.replace(' ', '%20')
    # Формируем полный URL: базовый адрес + команда + _ + год + .csv
    url = f"{base_url}{team_url}_{year}.csv"

    df = pd.read_csv(url) # Загружаем CSV-файл с составом команды
    df['team_name'] = team
    df['year'] = year # Добавляем колонки с названием команды и годом

    dfs.append(df) # Добавляем в список

teams_raw_df = pd.concat(dfs, ignore_index=True) # Объединяем все датафреймы в один

print("Raw teams shape:", teams_raw_df.shape)
teams_raw_df.head()

Raw teams shape: (170, 5)


,Unnamed: 0,Player Names,team_name,year,Players
0,0,Artstyle,Natus Vincere,2011,NaN
1,1,Dendi,Natus Vincere,2011,NaN
2,2,XBOCT,Natus Vincere,2011,NaN
3,3,Puppey,Natus Vincere,2011,NaN
4,4,LighTofHeaveN,Natus Vincere,2011,NaN


ПЕРВИЧНЫЙ АНАЛИЗ

In [38]:
# Смотрим, какие колонки у нас есть и есть ли пропуски
print(teams_raw_df.columns)
print(teams_raw_df.isnull().sum())

Index(['Unnamed: 0', 'Player Names', 'team_name', 'year', 'Players'], dtype='object')
Unnamed: 0        0
Player Names    137
team_name         0
year              0
Players          33
dtype: int64


**ЭТАП TRANSFORM**

Проблема: колонка с именами игроков может называться по-разному

Создаем функцию для унификации

In [39]:
def normalize_player_column(df):
    """
    Приводит колонку с именами игроков к единому стандарту.

    Проблема: в разных файлах колонка называется:
    - 'Player Names' (с пробелом)
    - 'Players'

    Решение: создаем единую колонку 'player_name'
    """

    df = df.copy() # Создаем копию, чтобы не менять исходные данные

    if 'Player Names' in df.columns:
        df['player_name'] = df['Player Names']
    elif 'Players' in df.columns:
        df['player_name'] = df['Players']
    else:
        df['player_name'] = None # Если ни одного варианта нет - ставим None

    return df

RELOAD WITH STANDARDIZATION

In [40]:
dfs = []

for _, row in winners_top3[['team_name', 'year']].drop_duplicates().iterrows():
    team = row['team_name']
    year = row['year']

    team_url = team.replace(' ', '%20')
    url = f"{base_url}{team_url}_{year}.csv"

    df = pd.read_csv(url)

    df = normalize_player_column(df) # ПРИМЕНЯЕМ НОРМАЛИЗАЦИЮ

    df['team_name'] = team
    df['year'] = year

    dfs.append(df)

teams_full_df = pd.concat(dfs, ignore_index=True) # Объединяем все нормализованные данные

print("After normalization:", teams_full_df.shape)
teams_full_df.head()

After normalization: (170, 6)


,Unnamed: 0,Player Names,player_name,team_name,year,Players
0,0,Artstyle,Artstyle,Natus Vincere,2011,NaN
1,1,Dendi,Dendi,Natus Vincere,2011,NaN
2,2,XBOCT,XBOCT,Natus Vincere,2011,NaN
3,3,Puppey,Puppey,Natus Vincere,2011,NaN
4,4,LighTofHeaveN,LighTofHeaveN,Natus Vincere,2011,NaN


INTERPRET TECHNICAL COLUMN — превращаем Unnamed: 0 в бизнес-атрибут

Интерпретация технического столбца позиции Unnamed: 0:

* присутствует во всех CSV
* отражает позицию внутри команды
* не является ошибкой данных

Переименую его и использую для определения роли

In [41]:
teams_full_df = teams_full_df.rename(
    columns={'Unnamed: 0': 'team_position'}
)

BUSINESS TRANSFORMATION — определяем роль участника


Формирование признака role

Из анализа структуры данных:

* позиции 0–4 основные игроки команды
* остальные строки менеджеры

In [42]:
# Применяем функцию lambda к каждой строке колонки team_position
# x in [0,1,2,3,4] проверяет, является ли позиция игровой
teams_full_df['role'] = teams_full_df['team_position'].apply(
    lambda x: 'player' if x in [0, 1, 2, 3, 4] else 'other'
)

ФОРМИРОВАНИЕ ФАКТ-ТАБЛИЦЫ

In [43]:
teams = teams_full_df[
    ['player_name', 'team_name', 'year', 'role']
].copy() # Оставляем только нужные колонки для будущей таблицы в DWH

teams = teams.drop_duplicates().reset_index(drop=True) # Удаляем возможные дубликаты и сбрасываем индексы

print("Final teams shape:", teams.shape)
teams.head()

Final teams shape: (170, 4)


,player_name,team_name,year,role
0,Artstyle,Natus Vincere,2011,player
1,Dendi,Natus Vincere,2011,player
2,XBOCT,Natus Vincere,2011,player
3,Puppey,Natus Vincere,2011,player
4,LighTofHeaveN,Natus Vincere,2011,player


**DATA QUALITY VALIDATION**

In [44]:
def validate_teams(df):
    """
    Проверка качества данных о составах команд.

    Проверяем:
    1. Отсутствие NULL в ключевых полях
    2. Уникальность записей (один игрок в команде в год - одна запись)
    3. Бизнес-правило: в команде минимум 5 игроков
    """

    errors = []


    # 1. Отсутствие NULL
    if df['player_name'].isna().any():
        errors.append("player_name contains NULLs")

    if df['team_name'].isna().any():
        errors.append("team_name contains NULLs")

    if df['year'].isna().any():
        errors.append("year contains NULLs")
   # 2. Проверка гранулярности
    dup = df.duplicated(subset=['player_name', 'team_name', 'year']).sum()
    if dup > 0:
        errors.append(f"Duplicate roster rows detected: {dup}")

    # 3. Минимальный размер команды
    # В Dota 2 в команде минимум 5 игроков
    # Фильтруем только игроков (не тренеров) и считаем их по командам и годам

    roster_size = df[df['role'] == 'player'].groupby(['team_name', 'year']).size()

    # Находим команды, где игроков меньше 5
    bad_teams = roster_size[roster_size < 5]

    if not bad_teams.empty:
        errors.append(f"Teams with <5 players detected: {bad_teams.index.tolist()}")

    # Итог
    if errors:
        for err in errors:
            logger.error(err)
        raise ValueError("Team roster validation failed")

    logger.info("Teams validation passed")

In [45]:
# Запускаем проверку
validate_teams(teams)

**СОХРАНЕНИЕ ПРОМЕЖУТОЧНЫХ РЕЗУЛЬТАТОВ (LOAD PREVIEW)**

In [ ]:
output_path = "/content/teams.csv"
hero_picks.to_csv(output_path, index=False)

logger.info(f"Data mart file saved to {output_path}")

***LOAD ГОТОВЫХ ДАННЫХ ИЗ COLAB***

In [ ]:
# Мы обработали все данные и сохранили их в CSV-файлы в папке /content/
# Теперь нужно скачать их на свой компьютер, чтобы потом загрузить в базу данных
from google.colab import files

files.download("/content/winners_top3.csv")
files.download("/content/teams.csv")
files.download("/content/country.csv")
files.download("/content/hero_picks.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>